In [1]:
import pandas as pd
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

import warnings
warnings.filterwarnings("ignore")

In [2]:
df  = pd.read_csv("SampleDataset.csv", encoding='latin-1')

In [3]:
possible_labels = df.TECHNOLOGY.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'PowerBI': 0, 'Azure Analysis Services': 1, 'Azure Data Factory': 2}

In [4]:
import collections

inverse_dict = collections.defaultdict(list)
for key, value in label_dict.items():
    inverse_dict[value].append(key)
print(inverse_dict)

defaultdict(<class 'list'>, {0: ['PowerBI'], 1: ['Azure Analysis Services'], 2: ['Azure Data Factory']})


In [5]:
sentence = input("Enter a sentence: ")

Enter a sentence: PowerBI - I am trying to create a dashboard. I used date slicer to filter dates to reflect data on the table. There are data in table till 2022 but the data slicer is moved beyond 4/31/2021 is not showing data in the tables


In [6]:
from transformers import BertForSequenceClassification
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

model.load_state_dict(torch.load('data volume/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

<All keys matched successfully>

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

encoded_sentence = tokenizer.encode_plus(
    sentence,
    add_special_tokens=True,
    max_length=256,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)


input_ids = encoded_sentence['input_ids'].to(device)
attention_mask = encoded_sentence['attention_mask'].to(device)

with torch.no_grad():
    model.eval()
    logits = model(input_ids, attention_mask=attention_mask)[0]

probs = torch.nn.functional.softmax(logits, dim=1)

predicted_label = torch.argmax(probs, dim=1).item()

predicted_class = inverse_dict[predicted_label]

print("Predicted Class:", predicted_class[0])

Predicted Class: PowerBI
